In [1]:
import numpy as np
import pandas as pd
import string
import random
import re
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [2]:
# https://www.kaggle.com/code/ihsncnkz/techcareer-project
df = pd.read_csv("magaza_yorumlari_duygu_analizi.csv", encoding="utf-16")


In [3]:
df.head()


,Görüş,Durum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz
1,hizli teslimat tesekkürler,Tarafsız
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11429 entries, 0 to 11428
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Görüş   11426 non-null  object
 1   Durum   11429 non-null  object
dtypes: object(2)
memory usage: 178.7+ KB


In [5]:
df.dropna(inplace=True)


In [6]:
df["Durum"].value_counts()


Olumlu      4252
Olumsuz     4237
Tarafsız    2937
Name: Durum, dtype: int64

In [7]:
df = df.loc[df["Durum"].isin(["Olumlu", "Olumsuz"])]


In [8]:
df["Durum"].value_counts()


Olumlu     4252
Olumsuz    4237
Name: Durum, dtype: int64

In [9]:
def etiketle(row):
    if row["Durum"] == "Olumlu":
        return 1
    return 0


In [10]:
df["label"] = df.apply(lambda row: etiketle(row), axis=1)


In [11]:
df.head()


,Görüş,Durum,label
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz,0
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu,1
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz,0
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz,0
6,kullanisli baya,Olumlu,1


In [12]:
for _ in range(10):
    idx = random.randint(0, len(df)-1)
    print(df.iloc[idx]["Görüş"])
    print("-" * 20)


SAKIN ALMAYIN !!!!

Aşağıdaki linkte detaylı olarak yazdım. Konu Tüketici Hakem Heyetinde. Sonucu bekliyorum.
--------------------
Beklentinin çok yüksek olmasın, Birincisi kargo şirketi berbat, başka
şirket yokmuş gibi yurtiçi kargo da niye ısrar ediliyor? Adamlar
ellerindeki ürünü eve getirmiyo, neyse ürüne gelince süper bir ürün
beklemeyin, başta paket çok zayıf, kargoda yırtılmış, ürün basit,
kaliteli birşey yapılmamış, ısıtıcı rezistanslar açıkta, geç pişiriyor
vb. Şeyler kısacası hayaller ve gerçekler farklı.
--------------------
ürün elime çok çabuk ulaştı çeyizime aldım inşallah pişman olmam daha kullanmadım
--------------------
Hemen yanibasinda kullanarak fiyat performans olarak cok iyi verim alabilirsiniz ancak 2 oda uzaklastiginizda cekmiyor cektiginde kopuyor. Genisletici ile kullaninca da surekli bir sorun oluyor tavsiye etmiyorum.
--------------------
Beklediğimden de kötü çıktı tavsiye etmiyorum
--------------------
Annem için ürünü satın aldım. yaşlı olduğu için çok te

In [13]:
stop_words = ["a", "acaba", "altı", "ama", "ancak", "artık", "asla", "aslında", "az", "b", "bana", "bazen", "bazı", "bazıları", "bazısı", "belki", "ben", "beni", "benim", "beş", "bile", "bir", "birçoğu", "birçok", "birçokları", "biri", "birisi", "birkaç", "birkaçı", "birşey", "birşeyi", "biz", "bize", "bizi", "bizim", "böyle", "böylece", "bu", "buna", "bunda", "bundan", "bunu", "bunun", "burada", "bütün", "c", "ç", "çoğu", "çoğuna", "çoğunu", "çok", "çünkü", "d", "da", "daha", "de", "değil", "demek", "diğer", "diğeri", "diğerleri", "diye", "dokuz", "dolayı", "dört", "e", "elbette", "en", "f", "fakat", "falan", "felan", "filan", "g", "gene", "gibi", "ğ", "h", "hâlâ", "hangi", "hangisi", "hani", "hatta", "hem", "henüz", "hep", "hepsi", "hepsine", "hepsini", "her", "her biri", "herkes", "herkese", "herkesi", "hiç", "hiç kimse", "hiçbiri", "hiçbirine", "hiçbirini", "ı", "i", "için", "içinde", "iki", "ile", "ise",
              "işte", "j", "k", "kaç", "kadar", "kendi", "kendine", "kendini", "ki", "kim", "kime", "kimi", "kimin", "kimisi", "l", "m", "madem", "mı", "mı", "mi", "mu", "mu", "mü", "mü", "n", "nasıl", "ne", "ne kadar", "ne zaman", "neden", "nedir", "nerde", "nerede", "nereden", "nereye", "nesi", "neyse", "niçin", "niye", "o", "on", "ona", "ondan", "onlar", "onlara", "onlardan", "onların", "onların", "onu", "onun", "orada", "oysa", "oysaki", "ö", "öbürü", "ön", "önce", "ötürü", "öyle", "p", "r", "rağmen", "s", "sana", "sekiz", "sen", "senden", "seni", "senin", "siz", "sizden", "size", "sizi", "sizin", "son", "sonra", "ş", "şayet", "şey", "şeyden", "şeye", "şeyi", "şeyler", "şimdi", "şöyle", "şu", "şuna", "şunda", "şundan", "şunlar", "şunu", "şunun", "t", "tabi", "tamam", "tüm", "tümü", "u", "ü", "üç", "üzere", "v", "var", "ve", "veya", "veyahut", "y", "ya", "ya da", "yani", "yedi", "yerine", "yine", "yoksa", "z", "zaten"]


In [14]:
def clean_text(text):
    text = " ".join(text.split())
    text = text.lower()
    text = text.replace("\\n", " ")
    text = re.sub("[0-9]+", "", text)
    ext = re.sub("%|(|)|-", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = [t for t in text.split() if t not in stop_words]
    return " ".join(text)


In [15]:
df["clean"] = df.apply(lambda row: clean_text(row["Görüş"]), axis=1)


In [16]:
for _ in range(10):
    idx = random.randint(0, len(df)-1)
    print(df.iloc[idx]["clean"])
    print("-" * 20)


ay buhar düğmesi bozuldu kesinlikle tavsiye etmiyorum
--------------------
kullanışsız
--------------------
balkonda bahçede kullandık başarılı
--------------------
geçen ay çeyizime almıştım kullanma fırsatım olmadı paketleme kargolama iyiydi fiyatı oldukça uyguna geldi indirimleri takip etmenizi öneririm parçalarımı tefal alıyordum bozmak istemedim düzeni mutfakta bence zaman iyisi tefal
--------------------
ürün beklediğimden küçük cok fazla ses vermemkteaçıkcası tavsiye etmiyorumkargo konusunda gayet iyidi
--------------------
kaliteli çeyiz muhtwsem
--------------------
beğenmedim aldım ürün verdiğim kargo
--------------------
ürün gereksiz şekilde uzun cm kısa olması gerekirmiş parmak okuyucu iyi çalışmıyor defa deniyorsunuz inceligi hafifliği güzel kamerası orta seviye cekiyor
--------------------
olumlu yönleri ürün istediğim emeği geçenlere teşekkürler
--------------------
beklediğim kaliteli
--------------------


In [17]:
df.head()


,Görüş,Durum,label,clean
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz,0,ses kalitesi ergonomisi rezalet sony olduğu al...
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu,1,ses olayı süpergece çalıştır sıkıntı yokkablo ...
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz,0,geldi bigün kullandık hemen bozoldu hiçtavsiye...
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz,0,kulaklığın sesi kaliteli aleti öve öve bitirem...
6,kullanisli baya,Olumlu,1,kullanisli baya


In [18]:
X = df["clean"].to_numpy()
y = df["label"].to_numpy()


In [19]:
X[:3]


array(['ses kalitesi ergonomisi rezalet sony olduğu aldım fiyatına çin replika ürün alsaydım iyiydi kesinlikle tavsiye etmiyorum',
       'ses olayı süpergece çalıştır sıkıntı yokkablo uzun işinizi rahat ettirirçekme olayı derece güzel içiniz rahat olsundiğerlerini saymıyorum',
       'geldi bigün kullandık hemen bozoldu hiçtavsiye etmem'],
      dtype=object)

In [20]:
print(len(X), len(y))


8489 8489


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [22]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [23]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)


DecisionTreeClassifier()

In [24]:
clf.score(X_test, y_test)


0.797997644287397

In [25]:
# modeli diske kaydetme
with open("tree.pkl", "wb") as f:
    pickle.dump(clf, f)

# vektörleştiriciyi diske kaydetme
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [26]:
# modeli diskten geri getirme
clf_loaded = None
with open("tree.pkl", "rb") as f:
    clf_loaded = pickle.load(f)

# vektörleştiriciyi diskten geri getirme
vectorizer_loaded = None
with open("vectorizer.pkl", "rb") as f:
    vectorizer_loaded = pickle.load(f)


In [28]:
# 1: Olumlu
# 0: olumsuz
test_str = [clean_text("Her şey çok güzeldi! Çok memnun kaldım.")]
test_str_vec = vectorizer_loaded.transform(test_str)
clf_loaded.predict(test_str_vec)


array([1])

In [30]:
cumleler = [
    "Çok kötü bit ürün... Asla tavsiye etmiyorum!",
    "Mükemmel ötesi muhteşemlikte :-)"
]

cumleler = [clean_text(s) for s in cumleler]

cumleler_vec = vectorizer_loaded.transform(cumleler)
clf_loaded.predict(cumleler_vec)

array([0, 1])